In [1]:
import json
import math
import pandas as pd
from collections import defaultdict, Counter
import re

# Load stopwords
with open("stopwords_id.txt", "r", encoding="utf-8") as f:
    STOPWORDS = set(line.strip().lower() for line in f if line.strip())

print(f"Loaded {len(STOPWORDS)} stopwords")

Loaded 761 stopwords


## 1. Load Data Index dan Metadata

In [2]:
# Load inverted index
with open("data/inverted_index.json", "r", encoding="utf-8") as f:
    inverted_index_raw = json.load(f)

# Convert to dict[str, dict[int, int]]
inverted_index = {}
for term, postings in inverted_index_raw.items():
    inverted_index[term] = {p["doc_id"]: p["tf"] for p in postings}

# Load document metadata
doc_meta = pd.read_csv("data/doc_meta.csv")
doc_meta = doc_meta.set_index("doc_id")

N = len(doc_meta)  # Total dokumen
vocab_size = len(inverted_index)

print(f"Total documents: {N}")
print(f"Vocabulary size: {vocab_size}")
print(doc_meta.head())

Total documents: 4700
Vocabulary size: 37662
                                                      url  \
doc_id                                                      
0       https://travel.kompas.com/read/2025/11/11/1900...   
1       https://travel.kompas.com/read/2025/11/11/1925...   
2       https://travel.kompas.com/read/2025/05/03/1333...   
3       https://travel.kompas.com/read/2025/11/10/1851...   
4       https://travel.kompas.com/read/2025/11/11/1806...   

                                                    title  doc_len  
doc_id                                                              
0       Unik! Kamar Hotel Bertema Kereta Api di Jepang...      581  
1       4 Rekomendasi Wisata di Banyuwangi, Cocok untu...      356  
2       Itinerary Seharian di Bromo Jawa Timur, dari S...      535  
3       Promo Hari Pahlawan 2025, TMII Beri Diskon 30 ...      486  
4       Gratis Tiket Wisata Trenggalek bagi Penumpang ...      375  


## 2. Preprocessing Query

Sama seperti preprocessing dokumen:
- Lowercase
- Tokenisasi
- Hapus stopwords
- Stemming (opsional dengan Sastrawi)

In [3]:
# Optional: Sastrawi stemmer
try:
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    print("[INFO] Sastrawi loaded, stemming AKTIF")
except ImportError:
    stemmer = None
    print("[INFO] Sastrawi not found, stemming NONAKTIF")

def preprocess_query(query: str) -> list:
    """Preprocessing query: lowercase, tokenize, remove stopwords, stem"""
    # Lowercase
    query = query.lower()
    
    # Remove special characters
    query = re.sub(r"[^a-z0-9\s]", " ", query)
    
    # Tokenize
    tokens = query.split()
    
    # Remove stopwords
    tokens = [t for t in tokens if t and t not in STOPWORDS]
    
    # Stemming
    if stemmer:
        tokens = [stemmer.stem(t) for t in tokens]
    
    return tokens

# Test
test_query = "wisata pantai di Bali yang indah"
print(f"Original: {test_query}")
print(f"Processed: {preprocess_query(test_query)}")

[INFO] Sastrawi loaded, stemming AKTIF
Original: wisata pantai di Bali yang indah
Processed: ['wisata', 'pantai', 'bal', 'indah']


## 3. Implementasi TF-IDF

**TF-IDF Formula:**

$$
\text{score}(q, d) = \sum_{t \in q} \text{tf}(t, d) \times \text{idf}(t)
$$

Dimana:
- $\text{tf}(t, d)$ = frekuensi term $t$ dalam dokumen $d$
- $\text{idf}(t) = \log\left(\frac{N + 1}{\text{df}(t) + 1}\right) + 1$

In [4]:
# Hitung IDF untuk semua term
idf = {}
for term, postings in inverted_index.items():
    df = len(postings)
    idf[term] = math.log((N + 1) / (df + 1)) + 1

def tfidf_search(query: str, top_k: int = 10):
    """Search menggunakan TF-IDF"""
    query_tokens = preprocess_query(query)
    
    if not query_tokens:
        return []
    
    # Hitung skor untuk setiap dokumen
    scores = defaultdict(float)
    
    for term in query_tokens:
        if term not in inverted_index:
            continue
        
        term_idf = idf[term]
        postings = inverted_index[term]
        
        for doc_id, tf in postings.items():
            scores[doc_id] += tf * term_idf
    
    # Sort by score
    ranked = sorted(scores.items(), key=lambda x: -x[1])[:top_k]
    
    # Format hasil
    results = []
    for doc_id, score in ranked:
        results.append({
            "doc_id": doc_id,
            "score": score,
            "title": doc_meta.loc[doc_id, "title"],
            "url": doc_meta.loc[doc_id, "url"]
        })
    
    return results

# Test TF-IDF
query = "wisata pantai Bali"
print(f"\nQuery: {query}")
print("="*80)
results = tfidf_search(query, top_k=5)
for i, res in enumerate(results, 1):
    print(f"{i}. [Score: {res['score']:.4f}] {res['title'][:70]}...")
    print(f"   {res['url']}")


Query: wisata pantai Bali
1. [Score: 241.0782] 15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
   https://travel.kompas.com/read/2022/05/05/110500627/15-tempat-wisata-bali-cocok-untuk-libur-panjang-?page=all#page2
2. [Score: 241.0782] 15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
   https://travel.kompas.com/read/2022/05/05/110500627/15-tempat-wisata-bali-cocok-untuk-libur-panjang-?page=all
3. [Score: 222.1399] 25 Wisata Bali yang Populer dan Unik, Pas buat Libur Panjang...
   https://travel.kompas.com/read/2022/05/16/131344027/25-wisata-bali-yang-populer-dan-unik-pas-buat-libur-panjang?page=all#page2
4. [Score: 167.8984] Panduan Wisata Lengkap ke Marine Safari Bali, Berapa Harga Tiketnya?...
   https://travel.kompas.com/read/2025/09/03/134511327/panduan-wisata-lengkap-ke-marine-safari-bali-berapa-harga-tiketnya?page=all#page2
5. [Score: 138.9712] 18 Wisata Pantai Gunungkidul yang Paling Terkenal, Cocok untuk Liburan...
   https://travel.kompas.com/read/2022/07/25/154700027/

## 4. Implementasi BM25

**BM25 Formula:**

$$
\text{score}(q, d) = \sum_{t \in q} \text{idf}(t) \cdot \frac{\text{tf}(t, d) \cdot (k_1 + 1)}{\text{tf}(t, d) + k_1 \cdot (1 - b + b \cdot \frac{|d|}{\text{avgdl}})}
$$

Dimana:
- $k_1$ = parameter tuning (default: 1.5)
- $b$ = parameter length normalization (default: 0.75)
- $|d|$ = panjang dokumen
- $\text{avgdl}$ = rata-rata panjang dokumen

In [5]:
# Hitung rata-rata panjang dokumen
avgdl = doc_meta["doc_len"].mean()
print(f"Average document length: {avgdl:.2f}")

def bm25_search(query: str, top_k: int = 10, k1: float = 1.5, b: float = 0.75):
    """Search menggunakan BM25"""
    query_tokens = preprocess_query(query)
    
    if not query_tokens:
        return []
    
    # Hitung skor untuk setiap dokumen
    scores = defaultdict(float)
    
    for term in query_tokens:
        if term not in inverted_index:
            continue
        
        term_idf = idf[term]
        postings = inverted_index[term]
        
        for doc_id, tf in postings.items():
            doc_len = doc_meta.loc[doc_id, "doc_len"]
            
            # BM25 formula
            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * (doc_len / avgdl))
            
            scores[doc_id] += term_idf * (numerator / denominator)
    
    # Sort by score
    ranked = sorted(scores.items(), key=lambda x: -x[1])[:top_k]
    
    # Format hasil
    results = []
    for doc_id, score in ranked:
        results.append({
            "doc_id": doc_id,
            "score": score,
            "title": doc_meta.loc[doc_id, "title"],
            "url": doc_meta.loc[doc_id, "url"]
        })
    
    return results

# Test BM25
query = "wisata pantai Bali"
print(f"\nQuery: {query}")
print("="*80)
results = bm25_search(query, top_k=5)
for i, res in enumerate(results, 1):
    print(f"{i}. [Score: {res['score']:.4f}] {res['title'][:70]}...")
    print(f"   {res['url']}")

Average document length: 401.01

Query: wisata pantai Bali
1. [Score: 15.2202] 15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
   http://travel.kompas.com/read/2022/05/05/110500627/15-tempat-wisata-bali-cocok-untuk-libur-panjang-
2. [Score: 15.2202] 15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
   https://travel.kompas.com/read/2022/05/05/110500627/15-tempat-wisata-bali-cocok-untuk-libur-panjang-?page=1
3. [Score: 15.2202] 15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
   https://travel.kompas.com/read/2022/05/05/110500627/15-tempat-wisata-bali-cocok-untuk-libur-panjang-
4. [Score: 15.0702] 5 Wisata Pantai Terkenal di Bali, Cocok untuk Liburan Akhir Tahun...
   https://travel.kompas.com/read/2024/11/19/190700727/5-wisata-pantai-terkenal-di-bali-cocok-untuk-liburan-akhir-tahun
5. [Score: 15.0229] Pesona Pantai Berawa di Bali, Indahnya Sunset dan Bisa Berselancar...
   http://travel.kompas.com/read/2022/08/27/190700427/pesona-pantai-berawa-di-bali-indahnya-sunset-dan-bisa-

## 5. Perbandingan TF-IDF vs BM25

Mari kita bandingkan hasil kedua algoritma untuk beberapa query.

In [6]:
def compare_algorithms(query: str, top_k: int = 5):
    """Bandingkan hasil TF-IDF dan BM25"""
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    print(f"{'='*80}\n")
    
    # TF-IDF
    print("\n🔍 TF-IDF Results:")
    print("-" * 80)
    tfidf_results = tfidf_search(query, top_k)
    for i, res in enumerate(tfidf_results, 1):
        print(f"{i}. [Score: {res['score']:.4f}]")
        print(f"   {res['title'][:75]}...")
    
    # BM25
    print("\n🎯 BM25 Results:")
    print("-" * 80)
    bm25_results = bm25_search(query, top_k)
    for i, res in enumerate(bm25_results, 1):
        print(f"{i}. [Score: {res['score']:.4f}]")
        print(f"   {res['title'][:75]}...")
    
    return tfidf_results, bm25_results

# Test dengan beberapa query
test_queries = [
    "wisata pantai Bali",
    "hotel murah Jakarta",
    "gunung Bromo sunrise",
    "kuliner Jogjakarta",
    "tempat wisata di Bandung"
]

for query in test_queries[:3]:  # Test 3 query pertama
    compare_algorithms(query, top_k=5)


Query: wisata pantai Bali


🔍 TF-IDF Results:
--------------------------------------------------------------------------------
1. [Score: 241.0782]
   15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
2. [Score: 241.0782]
   15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
3. [Score: 222.1399]
   25 Wisata Bali yang Populer dan Unik, Pas buat Libur Panjang...
4. [Score: 167.8984]
   Panduan Wisata Lengkap ke Marine Safari Bali, Berapa Harga Tiketnya?...
5. [Score: 138.9712]
   18 Wisata Pantai Gunungkidul yang Paling Terkenal, Cocok untuk Liburan Seko...

🎯 BM25 Results:
--------------------------------------------------------------------------------
1. [Score: 15.2202]
   15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
2. [Score: 15.2202]
   15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
3. [Score: 15.2202]
   15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
4. [Score: 15.0702]
   5 Wisata Pantai Terkenal di Bali, Cocok untuk Liburan Akhir Tahun...
5. [Score: 15.0229

## 6. Fungsi Search Interaktif

In [7]:
def interactive_search(query: str, algorithm: str = "bm25", top_k: int = 10):
    """
    Fungsi search yang lebih lengkap
    
    Args:
        query: Query pencarian
        algorithm: 'tfidf' atau 'bm25'
        top_k: Jumlah hasil yang ditampilkan
    """
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    print(f"Algorithm: {algorithm.upper()}")
    print(f"{'='*80}\n")
    
    if algorithm.lower() == "tfidf":
        results = tfidf_search(query, top_k)
    elif algorithm.lower() == "bm25":
        results = bm25_search(query, top_k)
    else:
        print("Algorithm tidak dikenal. Gunakan 'tfidf' atau 'bm25'")
        return []
    
    if not results:
        print("Tidak ada hasil yang ditemukan.")
        return []
    
    for i, res in enumerate(results, 1):
        print(f"{i}. [Score: {res['score']:.4f}]")
        print(f"   Title: {res['title']}")
        print(f"   URL: {res['url']}")
        print()
    
    return results

# Contoh penggunaan
interactive_search("wisata alam pegunungan", algorithm="bm25", top_k=5)


Query: wisata alam pegunungan
Algorithm: BM25

1. [Score: 12.6241]
   Title: Wisata Alam TN Gunung Merapi Pakai Pembayaran Nontunai dengan QRIS
   URL: http://travel.kompas.com/read/2025/02/01/120700227/wisata-alam-tn-gunung-merapi-pakai-pembayaran-nontunai-dengan-qris

2. [Score: 12.5568]
   Title: Kalitalang di Gunung Merapi adalah Ekowisata, Apa Itu?
   URL: https://travel.kompas.com/read/2023/10/23/093940127/kalitalang-di-gunung-merapi-adalah-ekowisata-apa-itu

3. [Score: 12.5568]
   Title: Kalitalang di Gunung Merapi adalah Ekowisata, Apa Itu?
   URL: http://travel.kompas.com/read/2023/10/23/093940127/kalitalang-di-gunung-merapi-adalah-ekowisata-apa-itu

4. [Score: 12.3970]
   Title: Gunung Padang di Cianjur, Punya Situs Megalitikum Terbesar di Asia Tenggara
   URL: https://travel.kompas.com/read/2022/09/23/180600427/gunung-padang-di-cianjur-punya-situs-megalitikum-terbesar-di-asia-tenggara

5. [Score: 12.3912]
   Title: Tarif Masuk TN Halimun Salak Turun 50 Persen, Jadi Segini H

[{'doc_id': 2358,
  'score': np.float64(12.624079044099117),
  'title': 'Wisata Alam TN Gunung Merapi Pakai Pembayaran Nontunai dengan QRIS',
  'url': 'http://travel.kompas.com/read/2025/02/01/120700227/wisata-alam-tn-gunung-merapi-pakai-pembayaran-nontunai-dengan-qris'},
 {'doc_id': 248,
  'score': np.float64(12.556768177532165),
  'title': 'Kalitalang di Gunung Merapi adalah Ekowisata, Apa Itu?',
  'url': 'https://travel.kompas.com/read/2023/10/23/093940127/kalitalang-di-gunung-merapi-adalah-ekowisata-apa-itu'},
 {'doc_id': 3832,
  'score': np.float64(12.556768177532165),
  'title': 'Kalitalang di Gunung Merapi adalah Ekowisata, Apa Itu?',
  'url': 'http://travel.kompas.com/read/2023/10/23/093940127/kalitalang-di-gunung-merapi-adalah-ekowisata-apa-itu'},
 {'doc_id': 2304,
  'score': np.float64(12.397007950244141),
  'title': 'Gunung Padang di Cianjur, Punya Situs Megalitikum Terbesar di Asia Tenggara',
  'url': 'https://travel.kompas.com/read/2022/09/23/180600427/gunung-padang-di-cia

## 7. Simpan Fungsi untuk Evaluasi

Fungsi-fungsi ini akan digunakan di notebook evaluasi.

In [8]:
# Export fungsi untuk digunakan di notebook lain
import pickle

search_functions = {
    "tfidf_search": tfidf_search,
    "bm25_search": bm25_search,
    "preprocess_query": preprocess_query,
    "inverted_index": inverted_index,
    "idf": idf,
    "doc_meta": doc_meta,
    "N": N,
    "avgdl": avgdl
}

with open("data/search_functions.pkl", "wb") as f:
    pickle.dump(search_functions, f)

print("\n✅ Fungsi search berhasil disimpan ke data/search_functions.pkl")


✅ Fungsi search berhasil disimpan ke data/search_functions.pkl


## 8. Ringkasan

Pada notebook ini kita telah:

1. ✅ Implementasi **TF-IDF** untuk pencarian dokumen
2. ✅ Implementasi **BM25** untuk pencarian dokumen
3. ✅ Membandingkan hasil kedua algoritma
4. ✅ Membuat fungsi interaktif untuk pencarian

**Perbedaan TF-IDF vs BM25:**
- **TF-IDF**: Simple, linear scoring. Baik untuk dokumen dengan panjang seragam.
- **BM25**: Lebih sophisticated dengan normalisasi panjang dokumen. Umumnya lebih baik untuk korpus dengan variasi panjang dokumen.

**Selanjutnya:**
Notebook `evaluation.ipynb` akan menghitung metrik evaluasi (Precision, Recall, F1, MAP) untuk kedua algoritma.